# Script takes in epochs with CSD applied and does STFT to get beta power

# Script Set Up

In [ ]:
%matplotlib inline

In [ ]:
!pip install mne==1.6

In [ ]:
import os
import numpy as np
from matplotlib import pyplot as plt
from scipy import stats
import pandas as pd
import seaborn as sns
import mne
import math
from scipy.io import loadmat
from scipy.fft import fft

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Load Epochs

In [ ]:
ID_list = ['P1', 'P3', 'P4', 'P7', 'P10', 'P12', 'P15', 'P17', 'P19',
           'P21', 'P22', 'P23', 'P24', 'P26', 'P27', 'P28', 'P29', 'P30']
ID = "P30"

fname = "/content/drive/MyDrive/Time_Reproduction/Epochs_Arrays_CSD/Encode/" + ID + "_Encode_All_CSD.mat"
encode_epochs = loadmat(fname)
encode_epochs = encode_epochs["encode_epochs_CSD"]
n_channels, n_samples_encode, n_epochs_encode = encode_epochs.shape

fname = "/content/drive/MyDrive/Time_Reproduction/Epochs_Arrays_CSD/Reproduction/" + ID + "_Reproduction_All_CSD.mat"
repo_epochs = loadmat(fname)
repo_epochs = repo_epochs["repo_epochs_CSD"]
n_channels, n_samples_repo, n_epochs_repo = repo_epochs.shape

fname = "/content/drive/MyDrive/Time_Reproduction/Epochs_Arrays_CSD/Response_Locked/" + ID + "_Response_Locked_All_CSD.mat"
RL_epochs = loadmat(fname)
RL_epochs = RL_epochs["response_locked_epochs_CSD"]
n_channels, n_samples_RL, n_epochs_RL = RL_epochs.shape

fname = "/content/drive/MyDrive/Time_Reproduction/Epochs_Arrays_CSD/Offset/" + ID + "_Offset_All_CSD.mat"
offset_epochs = loadmat(fname)
offset_epochs = offset_epochs["offset_epochs_CSD"]
n_channels, n_samples_offset, n_epochs_offset = offset_epochs.shape

(128, 564, 882)


# Also Load MNE Epochs for Times

In [ ]:
## MNE encode Epochs ##
fname = "/content/drive/MyDrive/Time_Reproduction/Epochs_CSD/Encode/" + ID + "_Encode_All_CSD_epo.fif"
MNE_encode_epochs = mne.read_epochs(fname, preload = True)
info = MNE_encode_epochs.average().info # use evoked so channels is 128

## MNE repo Epochs ##
fname = "/content/drive/MyDrive/Time_Reproduction/Epochs_CSD/Reproduction/" + ID + "_Reproduction_All_CSD_epo.fif"
MNE_repo_epochs = mne.read_epochs(fname, preload = True)

## MNE RL Epochs ##
fname = "/content/drive/MyDrive/Time_Reproduction/Epochs_CSD/Response_Locked/" + ID + "_RL_All_CSD_epo.fif"
MNE_RL_epochs = mne.read_epochs(fname, preload = True)

## MNE Offset Epochs ##
fname = "/content/drive/MyDrive/Time_Reproduction/Epochs_CSD/Offset/" + ID + "_Offset_All_MAT_CSD_epo.fif"
MNE_offset_epochs = mne.read_epochs(fname, preload = True)
info = MNE_offset_epochs.average().info

Reading /content/drive/MyDrive/Time_Reproduction/Epochs_CSD/Offset/P30_Offset_All_MAT_CSD_epo.fif ...
    Found the data of interest:
        t =     -99.61 ...    1000.00 ms
        0 CTF compensation matrices available
Adding metadata with 9 columns
882 matching events found
No baseline correction applied
0 projection items activated


# Set up for STFT

## Define Sample Rate and Frequencies of Interest
8-30Hz for Mu Beta

In [ ]:
fs = 512; # Define sampling rate (fs)
fs_beta = [8, 30] # Define frequencies we are interested in
fs_beta_only = [15, 30]
fs_mu_only = [8, 13] #?
fs_alpha = [8, 12] ## added 09/09 ## dave does 8-14 but over a much bigger window (707ms)
SSVEP_freq = 0 # need to figure out what this is
beta_time = []
beta_time_repo = []
beta_time_RL = []

beta_time_offset = []

## Define time parameters of your epochs

In [ ]:
## Encode epoch ##
epoch_limits_ms = (MNE_encode_epochs.tmin * 1000, MNE_encode_epochs.tmax * 1000) # start and end of your epoch in ms
t = np.linspace(epoch_limits_ms[0], epoch_limits_ms[1], n_samples_encode) # times in ms
ts = t / 1000 * fs # times in samples

## Repo epoch ##
epoch_limits_ms = (MNE_repo_epochs.tmin * 1000, MNE_repo_epochs.tmax * 1000) # start and end of your epoch in ms
t_repo = np.linspace(epoch_limits_ms[0], epoch_limits_ms[1], n_samples_repo) # times in ms
ts_repo = t_repo / 1000 * fs # times in samples

## RL epoch ##
epoch_limits_ms_RL = (MNE_RL_epochs.tmin * 1000, MNE_RL_epochs.tmax * 1000) # start and end of your epoch in ms
t_RL = np.linspace(epoch_limits_ms_RL[0], epoch_limits_ms_RL[1], n_samples_RL) # times in ms
ts_RL = t_RL / 1000 * fs # times in samples

## Offset epoch ##
epoch_limits_ms_offset = (MNE_offset_epochs.tmin * 1000, MNE_offset_epochs.tmax * 1000) # start and end of your epoch in ms
t_offset = np.linspace(epoch_limits_ms_offset[0], epoch_limits_ms_offset[1], n_samples_offset) # times in ms
ts_offset = t_offset / 1000 * fs # times in samples

## Define window size for (each) fft - should be long enough to capture 2 or 3 cycles of the lowest frequency

In [ ]:
min_freq = fs_beta[0]
one_min_freq_cycle_time = 1000 / min_freq
n_cycles = 3 # define the number of cycles of min freq that you want
stftlen_beta_t = one_min_freq_cycle_time * n_cycles # define window size in ms

stftlen_beta = stftlen_beta_t / 1000 * fs # convert from time to samples
stftlen_beta = int(stftlen_beta)

375.0


## Define the step size for moving your sliding window

In [ ]:
skip_step_beta_ms = 25
skip_step_beta = round(skip_step_beta_ms*fs/1000)

## Get frequency bins you will get running the FT in each window AND Get list of centre times of each window

In [ ]:
cc = 0

for tt in range(0, int(len(ts) - stftlen_beta), skip_step_beta):
  # chop out your window
  tf = np.arange(tt, tt+stftlen_beta)
  tf = np.array(tf, dtype = int)
  nfft = len(tf) # number of timepoints in time window

  # get your frequency bins
  freq_temp_beta = (np.arange(0,math.ceil((nfft+1)/2))*fs/nfft)

  # get the mean of that window in time. this is the new centre point of this sample in your new time vector.
  beta_time.append(np.mean(t[tf]))

  # shlide on
  cc += 1

## Do the same thing for the repo epoch

In [ ]:
cc = 0

for ttrepo in range(0, int(len(ts_repo) - stftlen_beta), skip_step_beta):
  # chop out your window
  tfrepo = np.arange(ttrepo, ttrepo+stftlen_beta)
  tfrepo = np.array(tfrepo, dtype = int)
  nfftrepo = len(tfrepo) # number of timepoints in time window

  # get your frequency bins
  freq_temp_beta_repo = (np.arange(0,math.ceil((nfftrepo+1)/2))*fs/nfftrepo)

  # get the mean of that window in time. this is the new centre point of this sample in your new time vector.
  beta_time_repo.append(np.mean(t_repo[tfrepo]))

  # shlide on
  cc += 1

## Do the same thing for the RL epoch

In [ ]:
cc = 0

for ttr in range(0, int(len(ts_RL) - stftlen_beta), skip_step_beta):
  # chop out your window
  tfr = np.arange(ttr, ttr+stftlen_beta)
  tfr = np.array(tfr, dtype = int)
  nfftr = len(tfr) # number of timepoints in time window

  # get your frequency bins
  freq_temp_beta_RL = (np.arange(0,math.ceil((nfftr+1)/2))*fs/nfftr)

  # get the mean of that window in time. this is the new centre point of this sample in your new time vector.
  beta_time_RL.append(np.mean(t_RL[tfr]))

  # shlide on
  cc += 1

## Do same thing for offset epoch

In [ ]:
cc = 0

for ttoffset in range(0, int(len(ts_offset) - stftlen_beta), skip_step_beta):
  # chop out your window
  tfoffset = np.arange(ttoffset, ttoffset+stftlen_beta)
  tfoffset = np.array(tfoffset, dtype = int)
  nfftoffset = len(tfoffset) # number of timepoints in time window

  # get your frequency bins
  freq_temp_beta_offset = (np.arange(0,math.ceil((nfftoffset+1)/2))*fs/nfftoffset)

  # get the mean of that window in time. this is the new centre point of this sample in your new time vector.
  beta_time_offset.append(np.mean(t_offset[tfoffset]))

  # shlide on
  cc += 1

192
364


# Computing STFT

## Encode Epochs

In [ ]:
beta = np.zeros((n_channels, len(beta_time),  n_epochs_encode)) # will contain target beta power for each channel, timebin, trial
beta_only = np.zeros((n_channels, len(beta_time),  n_epochs_encode)) # will contain target beta power for each channel, timebin, trial
mu_only = np.zeros((n_channels, len(beta_time),  n_epochs_encode)) # will contain target beta power for each channel, timebin, trial

erp = encode_epochs
cc = 0 # change to 0 cos python

n_total_freqs = stftlen_beta
n_freqs = len(np.arange(0, int(np.ceil((n_total_freqs + 1)/2))))
fft_all = np.zeros((n_channels, n_freqs, len(beta_time)))

print('Calculating encode beta...')

for tt in range(0, int(erp.shape[1]) - stftlen_beta, skip_step_beta):
  tf = np.arange(tt, tt+stftlen_beta)
  #print(tf.shape) # same length as stftlen_beta
  ep = erp[:, tf, :] # all channels, timepoints for fft, all epochs
  nfft = ep.shape[1] # not really needed to be defined again

  fftx = np.abs(fft(ep, axis = 1)) / (stftlen_beta/2) # computing fft
  fftx = fftx[:,0:int(np.ceil((nfft + 1)/2)),:] # getting frequencies you can have in your fft
  fftx_shape = fftx.shape

  # we only want power in the beta band, get those frequencies
  ind = np.argwhere((freq_temp_beta>=fs_beta[0]) & (freq_temp_beta<=fs_beta[1]) & (np.round(freq_temp_beta) != SSVEP_freq))
  ind_beta_only = np.argwhere((freq_temp_beta>=fs_beta_only[0]) & (freq_temp_beta<=fs_beta_only[1]) & (np.round(freq_temp_beta) != SSVEP_freq))
  ind_mu_only = np.argwhere((freq_temp_beta>=fs_mu_only[0]) & (freq_temp_beta<=fs_mu_only[1]) & (np.round(freq_temp_beta) != SSVEP_freq))


  # putting fft from this time window into bigger beta array with all time windows
  beta[:, cc, :] = np.mean(fftx[:,ind,:], axis = 1)[:, 0, :]
  beta_only[:, cc, :] = np.mean(fftx[:,ind_beta_only,:], axis = 1)[:, 0, :]
  mu_only[:, cc, :] = np.mean(fftx[:,ind_mu_only,:], axis = 1)[:, 0, :]

  # Getting all frequencies - need to average across epochs first otherwise array too big for google colab
  average_fftx = np.nanmean(fftx, axis = 2)
  fft_all[:, :, cc] = average_fftx

  cc+=1

## Repo Epochs

In [ ]:
beta_repo = np.zeros((n_channels, len(beta_time_repo),  n_epochs_repo)) # will contain target beta power for each channel, timebin, trial
beta_only_repo = np.zeros((n_channels, len(beta_time_repo),  n_epochs_repo))
mu_only_repo = np.zeros((n_channels, len(beta_time_repo),  n_epochs_repo))

fft_all_repo = np.zeros((n_channels, n_freqs, len(beta_time_repo)))

erprepo = repo_epochs
cc = 0 # change to 0 cos python
print('Calculating repo beta...')

for ttr in range(0, int(erprepo.shape[1]) - stftlen_beta, skip_step_beta):
  tfr = np.arange(ttr, ttr+stftlen_beta)

  eprepo = erprepo[:, tfr, :] # all channels, timepoints for fft, all epochs
  nfftr = eprepo.shape[1] # not really needed to be defined again

  fftx = np.abs(fft(eprepo, axis = 1)) / (stftlen_beta/2) # computing fft
  fftx = fftx[:,0:int(np.ceil((nfftr + 1)/2)),:] # getting frequencies you can have in your fft
  fftx_shape = fftx.shape

  # we only want power in the beta band, get those frequencies
  ind = np.argwhere((freq_temp_beta_repo>=fs_beta[0]) & (freq_temp_beta_repo<=fs_beta[1]) & (np.round(freq_temp_beta_repo) != SSVEP_freq))
  ind_beta_only = np.argwhere((freq_temp_beta_repo>=fs_beta_only[0]) & (freq_temp_beta_repo<=fs_beta_only[1]) & (np.round(freq_temp_beta_repo) != SSVEP_freq))
  ind_mu_only = np.argwhere((freq_temp_beta_repo>=fs_mu_only[0]) & (freq_temp_beta_repo<=fs_mu_only[1]) & (np.round(freq_temp_beta_repo) != SSVEP_freq))

  # putting fft from this time window into bigger beta array with all time windows
  beta_repo[:, cc, :] = np.mean(fftx[:,ind,:], axis = 1)[:, 0, :]
  beta_only_repo[:, cc, :] = np.mean(fftx[:,ind_beta_only,:], axis = 1)[:, 0, :]
  mu_only_repo[:, cc, :] = np.mean(fftx[:,ind_mu_only,:], axis = 1)[:, 0, :]

  # Getting all frequencies - need to average across epochs first otherwise array too big for google colab
  average_fftx = np.nanmean(fftx, axis = 2)
  fft_all_repo[:, :, cc] = average_fftx

  cc+=1

## Response-Locked Epochs

In [ ]:
beta_RL = np.zeros((n_channels, len(beta_time_RL),  n_epochs_RL)) # will contain target beta power for each channel, timebin, trial
beta_only_RL = np.zeros((n_channels, len(beta_time_RL),  n_epochs_RL))
mu_only_RL = np.zeros((n_channels, len(beta_time_RL),  n_epochs_RL))

fft_all_RL = np.zeros((n_channels, n_freqs, len(beta_time_RL)))

erpRL = RL_epochs
cc = 0 # change to 0 cos python
print('Calculating RL beta...')

for ttr in range(0, int(erpRL.shape[1]) - stftlen_beta, skip_step_beta):
  tfr = np.arange(ttr, ttr+stftlen_beta)

  epRL = erpRL[:, tfr, :] # all channels, timepoints for fft, all epochs
  nfftr = epRL.shape[1] # not really needed to be defined again

  fftx = np.abs(fft(epRL, axis = 1)) / (stftlen_beta/2) # computing fft
  fftx = fftx[:,0:int(np.ceil((nfftr + 1)/2)),:] # getting frequencies you can have in your fft
  fftx_shape = fftx.shape

  # we only want power in the beta band, get those frequencies
  ind = np.argwhere((freq_temp_beta_RL>=fs_beta[0]) & (freq_temp_beta_RL<=fs_beta[1]) & (np.round(freq_temp_beta_RL) != SSVEP_freq))
  ind_beta_only = np.argwhere((freq_temp_beta_RL>=fs_beta_only[0]) & (freq_temp_beta_RL<=fs_beta_only[1]) & (np.round(freq_temp_beta_RL) != SSVEP_freq))
  ind_mu_only = np.argwhere((freq_temp_beta_RL>=fs_mu_only[0]) & (freq_temp_beta_RL<=fs_mu_only[1]) & (np.round(freq_temp_beta_RL) != SSVEP_freq))

  # putting fft from this time window into bigger beta array with all time windows
  beta_RL[:, cc, :] = np.mean(fftx[:,ind,:], axis = 1)[:, 0, :]
  beta_only_RL[:, cc, :] = np.mean(fftx[:,ind_beta_only,:], axis = 1)[:, 0, :]
  mu_only_RL[:, cc, :] = np.mean(fftx[:,ind_mu_only,:], axis = 1)[:, 0, :]

  # Getting all frequencies - need to average across epochs first otherwise array too big for google colab
  average_fftx = np.nanmean(fftx, axis = 2)
  fft_all_RL[:, :, cc] = average_fftx
  cc+=1

## Offset Epochs

In [ ]:
beta_offset = np.zeros((n_channels, len(beta_time_offset),  n_epochs_offset)) # will contain target beta power for each channel, timebin, trial
beta_only_offset = np.zeros((n_channels, len(beta_time_offset),  n_epochs_offset))
mu_only_offset = np.zeros((n_channels, len(beta_time_offset),  n_epochs_offset))

n_total_freqs = stftlen_beta
n_freqs = len(np.arange(0, int(np.ceil((n_total_freqs + 1)/2))))
fft_all_offset = np.zeros((n_channels, n_freqs, len(beta_time_offset)))

erpoffset = offset_epochs
cc = 0 # change to 0 cos python
print('Calculating offset beta...')

for ttr in range(0, int(erpoffset.shape[1]) - stftlen_beta, skip_step_beta):
  tfr = np.arange(ttr, ttr+stftlen_beta)

  epoffset = erpoffset[:, tfr, :] # all channels, timepoints for fft, all epochs
  nfftr = epoffset.shape[1] # not really needed to be defined again

  fftx = np.abs(fft(epoffset, axis = 1)) / (stftlen_beta/2) # computing fft
  fftx = fftx[:,0:int(np.ceil((nfftr + 1)/2)),:] # getting frequencies you can have in your fft
  fftx_shape = fftx.shape

  # we only want power in the beta band, get those frequencies
  ind = np.argwhere((freq_temp_beta_offset>=fs_beta[0]) & (freq_temp_beta_offset<=fs_beta[1]) & (np.round(freq_temp_beta_offset) != SSVEP_freq))
  ind_beta_only = np.argwhere((freq_temp_beta_offset>=fs_beta_only[0]) & (freq_temp_beta_offset<=fs_beta_only[1]) & (np.round(freq_temp_beta_offset) != SSVEP_freq))
  ind_mu_only = np.argwhere((freq_temp_beta_offset>=fs_mu_only[0]) & (freq_temp_beta_offset<=fs_mu_only[1]) & (np.round(freq_temp_beta_offset) != SSVEP_freq))

  # putting fft from this time window into bigger beta array with all time windows
  beta_offset[:, cc, :] = np.mean(fftx[:,ind,:], axis = 1)[:, 0, :]
  beta_only_offset[:, cc, :] = np.mean(fftx[:,ind_beta_only,:], axis = 1)[:, 0, :]
  mu_only_offset[:, cc, :] = np.mean(fftx[:,ind_mu_only,:], axis = 1)[:, 0, :]

  # Getting all frequencies - need to average across epochs first otherwise array too big for google colab
  average_fftx = np.nanmean(fftx, axis = 2)
  fft_all_offset[:, :, cc] = average_fftx

  cc+=1

(128, 564, 882)
564
Calculating offset beta...


# Saving Data for each participant
Have array of n_channels, n_time_windows, n_epochs

Might want to save as MNE array too?

In [ ]:
## Saving All frequencies averaged across epochs
fname = "/content/drive/MyDrive/Time_Reproduction/Beta/" + ID + "_fft_all_encode.npy"
np.save(fname, fft_all)

fname = "/content/drive/MyDrive/Time_Reproduction/Beta/" + ID + "_fft_all_repo.npy"
np.save(fname, fft_all_repo)

fname = "/content/drive/MyDrive/Time_Reproduction/Beta/" + ID + "_fft_all_RL.npy"
np.save(fname, fft_all_RL)

In [ ]:
fname = "/content/drive/MyDrive/Time_Reproduction/Beta/" + ID + "_beta_encode.npy"
np.save(fname, beta)

fname = "/content/drive/MyDrive/Time_Reproduction/Beta/" + ID + "_beta_repo.npy"
np.save(fname, beta_repo)

fname = "/content/drive/MyDrive/Time_Reproduction/Beta/" + ID + "_beta_RL.npy"
np.save(fname, beta_RL)

In [ ]:
fname = "/content/drive/MyDrive/Time_Reproduction/Beta/" + ID + "_beta_only_encode.npy"
np.save(fname, beta_only)

fname = "/content/drive/MyDrive/Time_Reproduction/Beta/" + ID + "_beta_only_repo.npy"
np.save(fname, beta_only_repo)

fname = "/content/drive/MyDrive/Time_Reproduction/Beta/" + ID + "_beta_only_RL.npy"
np.save(fname, beta_only_RL)

In [ ]:
fname = "/content/drive/MyDrive/Time_Reproduction/Beta/" + ID + "_beta_offset.npy"
np.save(fname, beta_offset)

# Convert to MNE object before saving
Can add in metadata for the epochs

# Setting Up Epochs Array for MNE
Need to change dimensions to form of n_epochs, n_channels, n_freqs, n_times

Does unit need to be changed?

In [ ]:
## Encode ##
beta = np.moveaxis(beta, -1, 0)
beta = beta[:, :, :, np.newaxis]
beta = np.moveaxis(beta, -1, -2)

## repo ##
beta_repo = np.moveaxis(beta_repo, -1, 0)
beta_repo = beta_repo[:, :, :, np.newaxis]
beta_repo = np.moveaxis(beta_repo, -1, -2)

## RL ##
beta_RL = np.moveaxis(beta_RL, -1, 0)
beta_RL = beta_RL[:, :, :, np.newaxis]
beta_RL = np.moveaxis(beta_RL, -1, -2)

## Offset
beta_offset = np.moveaxis(beta_offset, -1, 0)
beta_offset = beta_offset[:, :, :, np.newaxis]
beta_offset = np.moveaxis(beta_offset, -1, -2)

(128, 29, 882)
(882, 128, 29)
(882, 128, 29, 1)
(882, 128, 1, 29)


In [ ]:
## Encode ##
beta_only = np.moveaxis(beta_only, -1, 0)
beta_only = beta_only[:, :, :, np.newaxis]
beta_only = np.moveaxis(beta_only, -1, -2)

## Repo ##
beta_only_repo = np.moveaxis(beta_only_repo, -1, 0)
beta_only_repo = beta_only_repo[:, :, :, np.newaxis]
beta_only_repo = np.moveaxis(beta_only_repo, -1, -2)

## RL ##
beta_only_RL = np.moveaxis(beta_only_RL, -1, 0)
beta_only_RL = beta_only_RL[:, :, :, np.newaxis]
beta_only_RL = np.moveaxis(beta_only_RL, -1, -2)

In [ ]:
## Encode ##
mu_only = np.moveaxis(mu_only, -1, 0)
mu_only = mu_only[:, :, :, np.newaxis]
mu_only = np.moveaxis(mu_only, -1, -2)

## repo ##
mu_only_repo = np.moveaxis(mu_only_repo, -1, 0)
mu_only_repo = mu_only_repo[:, :, :, np.newaxis]
mu_only_repo = np.moveaxis(mu_only_repo, -1, -2)

## RL ##
mu_only_RL = np.moveaxis(mu_only_RL, -1, 0)
mu_only_RL = mu_only_RL[:, :, :, np.newaxis]
mu_only_RL = np.moveaxis(mu_only_RL, -1, -2)

## Create EpochsTFR (epochs time frequency) object

### Encode Epochs

In [ ]:
info = info
data = beta
times = np.array(beta_time) / 1000 # middle time points
freqs = np.array([int((fs_beta[0] + fs_beta[1]) / 2)]) # need to label the frequency something so just taking the average
method = "STFT"
events = MNE_encode_epochs.events
metadata = MNE_encode_epochs.metadata

event_id = {"Encode/600":51, "Encode/650":52, "Encode/700":53,
            "Encode/750":54, "Encode/800":55, "Encode/850":56,
            "Encode/900":57, "Encode/950":58, "Encode/1000":59}

beta_epochs = mne.time_frequency.EpochsTFR(info = info, data = data, times = times, freqs = freqs, method = method,
                                           events = events, event_id = event_id, metadata = metadata)

data = beta_only
freqs = np.array([int((fs_beta_only[0] + fs_beta_only[1]) / 2)])

beta_only_epochs = mne.time_frequency.EpochsTFR(info = info, data = data, times = times, freqs = freqs, method = method,
                                           events = events, event_id = event_id, metadata = metadata)

data = mu_only
freqs = np.array([int((fs_mu_only[0] + fs_mu_only[1]) / 2)])

mu_only_epochs = mne.time_frequency.EpochsTFR(info = info, data = data, times = times, freqs = freqs, method = method,
                                           events = events, event_id = event_id, metadata = metadata)

### Repo Epochs

In [ ]:
info = info
data = beta_repo
times = np.array(beta_time_repo) / 1000 # middle time points
freqs = np.array([int((fs_beta[0] + fs_beta[1]) / 2)]) # need to label the frequency something so just taking the average
method = "STFT"
events = MNE_repo_epochs.events
metadata = MNE_repo_epochs.metadata

event_id = {"Reproduction/600":151, "Reproduction/650":152, "Reproduction/700":153,
            "Reproduction/750":154, "Reproduction/800":155, "Reproduction/850":156,
            "Reproduction/900":157, "Reproduction/950":158, "Reproduction/1000":159}

beta_epochs_repo = mne.time_frequency.EpochsTFR(info = info, data = data, times = times, freqs = freqs, method = method,
                                           events = events, event_id = event_id, metadata = metadata)

data = beta_only_repo
freqs = np.array([int((fs_beta_only[0] + fs_beta_only[1]) / 2)])

beta_only_epochs_repo = mne.time_frequency.EpochsTFR(info = info, data = data, times = times, freqs = freqs, method = method,
                                           events = events, event_id = event_id, metadata = metadata)

data = mu_only_repo
freqs = np.array([int((fs_mu_only[0] + fs_mu_only[1]) / 2)])

mu_only_epochs_repo = mne.time_frequency.EpochsTFR(info = info, data = data, times = times, freqs = freqs, method = method,
                                           events = events, event_id = event_id, metadata = metadata)

### RL Epochs

In [ ]:
info = info
data = beta_RL
times = np.array(beta_time_RL) / 1000 # middle time points
freqs = np.array([int((fs_beta[0] + fs_beta[1]) / 2)]) # need to label the frequency something so just taking the average
method = "STFT"
events = MNE_RL_epochs.events
metadata = MNE_RL_epochs.metadata

event_id = {"Response/600":201, "Response/650":202, "Response/700":203,
            "Response/750":204, "Response/800":205, "Response/850":206,
            "Response/900":207, "Response/950":208, "Response/1000":209}

beta_epochs_RL = mne.time_frequency.EpochsTFR(info = info, data = data, times = times, freqs = freqs, method = method,
                                           events = events, event_id = event_id, metadata = metadata)

data = beta_only_RL
freqs = np.array([int((fs_beta_only[0] + fs_beta_only[1]) / 2)])

beta_only_epochs_RL = mne.time_frequency.EpochsTFR(info = info, data = data, times = times, freqs = freqs, method = method,
                                           events = events, event_id = event_id, metadata = metadata)

data = mu_only_RL
freqs = np.array([int((fs_mu_only[0] + fs_mu_only[1]) / 2)])

mu_only_epochs_RL = mne.time_frequency.EpochsTFR(info = info, data = data, times = times, freqs = freqs, method = method,
                                           events = events, event_id = event_id, metadata = metadata)

### Offset Epochs

In [ ]:
info = info
data = beta_offset
times = np.array(beta_time_offset) / 1000 # middle time points
freqs = np.array([int((fs_beta[0] + fs_beta[1]) / 2)]) # need to label the frequency something so just taking the average
method = "STFT"
events = MNE_offset_epochs.events
metadata = MNE_offset_epochs.metadata

event_id = {"Offset/600":101, "Offset/650":102, "Offset/700":103,
            "Offset/750":104, "Offset/800":105, "Offset/850":106,
            "Offset/900":107, "Offset/950":108, "Offset/1000":109}

beta_epochs_offset = mne.time_frequency.EpochsTFR(info = info, data = data, times = times, freqs = freqs, method = method,
                                           events = events, event_id = event_id, metadata = metadata)

Adding metadata with 9 columns
<EpochsTFR | time : [0.086914, 0.797852], freq : [19.000000, 19.000000], epochs : 882, channels : 128, ~25.2 MB>


## Save Epochs

In [ ]:
pip install h5io --upgrade

In [ ]:
fname = "/content/drive/MyDrive/Time_Reproduction/Beta_Epochs/" + ID + "_encode_beta_epochs-tfr.h5"
beta_epochs.save(fname, overwrite = True)

fname = "/content/drive/MyDrive/Time_Reproduction/Beta_Epochs/" + ID + "_repo_beta_epochs-tfr.h5"
beta_epochs_repo.save(fname, overwrite = True)

fname = "/content/drive/MyDrive/Time_Reproduction/Beta_Epochs/" + ID + "_RL_beta_epochs-tfr.h5"
beta_epochs_RL.save(fname, overwrite = True)

In [ ]:
fname = "/content/drive/MyDrive/Time_Reproduction/Beta_Epochs/" + ID + "_encode_beta_only_epochs-tfr.h5"
beta_only_epochs.save(fname, overwrite = True)

fname = "/content/drive/MyDrive/Time_Reproduction/Beta_Epochs/" + ID + "_repo_beta_only_epochs-tfr.h5"
beta_only_epochs_repo.save(fname, overwrite = True)

fname = "/content/drive/MyDrive/Time_Reproduction/Beta_Epochs/" + ID + "_RL_beta_only_epochs-tfr.h5"
beta_only_epochs_RL.save(fname, overwrite = True)

In [ ]:
fname = "/content/drive/MyDrive/Time_Reproduction/Beta_Epochs/" + ID + "_encode_mu_only_epochs-tfr.h5"
mu_only_epochs.save(fname, overwrite = True)

fname = "/content/drive/MyDrive/Time_Reproduction/Beta_Epochs/" + ID + "_repo_mu_only_epochs-tfr.h5"
mu_only_epochs_repo.save(fname, overwrite = True)

fname = "/content/drive/MyDrive/Time_Reproduction/Beta_Epochs/" + ID + "_RL_mu_only_epochs-tfr.h5"
mu_only_epochs_RL.save(fname, overwrite = True)

In [ ]:
fname = "/content/drive/MyDrive/Time_Reproduction/Beta_Epochs/" + ID + "_Offset_beta_epochs-tfr.h5"
beta_epochs_offset.save(fname, overwrite = True)